In [234]:
__author__ = 'pratyush'

from bs4 import BeautifulSoup
from abbreviations import schwartz_hearst as sh
import urllib
import bson
import html2text
import numpy as np
import pandas as pd
import json
import copy
import unicodedata
from collections import defaultdict, Counter, OrderedDict
import re
import pickle
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
import random
import Levenshtein as lvst
import numpy as np
import wikipedia as wiki
from IPython import display,html
import json
import urllib
from IPython.core.display import HTML
from Normalisation_Helper import first_letters,normalize_unicode_to_ascii,get_1st_letters_match

In [235]:
data_location = json.load(open("corpus/location_norm2raw_v1.json"))
data_company = json.load(open("corpus/company_norm2raw_v1.json"))

In [241]:
loc_keys = list(data_location.keys())
comp_keys = list(data_company.keys())

In [242]:
loc_keys

["Coeur d' Alene, ID", 'San Francisco, CA', 'Washington. D.C.', 'KOOTENAI County, Idaho', 'Los Angeles, California', 'New York, New York', "St. Louis'", 'The United States', 'HARRISBURG, PA', 'San Jose, CA', 'Seattle, Wa', 'Interstate 70', 'Interstate 630', 'South Bend, Indiana', 'Crystal Lane', 'Atlanta, GA', 'WASHINGTON, D. C', 'Kansas City, Mo.', 'Chicago, Illinois', 'LOS ANGELES, CA', 'TAMPA, Florida', 'Brunswick, GA', 'RALEIGH, NORTH CAROLINA', 'HOUSTON, Texas', 'The United States of America', 'Sri Lanka', 'King of Prussia, PA', 'ASIA PACIFIC', "Los Angeles'", 'Shenzhen, China', 'Denver County', 'Shanghai, CHINA', 'IRVINE, California', 'London, United Kingdom', 'MADISON, Wisconsin', 'Jacksonville, Florida', 'Pensacola, Florida', 'WASHINGTON: STATE', 'Columbus, Ohio', 'Austin, Texas', 'Philadelphia, PA', 'Portsmouth, VA', 'Columbia, MD', 'NEW YORK, NY', 'Guangdong, CHINA', 'Wuhan,  China', 'THE PHILIPPINES', 'St. Paul, MN', "COEUR D' ALENE", 'MADISON, WI', 'Phoenix, Arizona', 'PHOE

In [243]:
loc_keys = loc_keys[:10]
loc_keys

["Coeur d' Alene, ID", 'San Francisco, CA', 'Washington. D.C.', 'KOOTENAI County, Idaho', 'Los Angeles, California', 'New York, New York', "St. Louis'", 'The United States', 'HARRISBURG, PA', 'San Jose, CA']

In [250]:
def get_summary(key):
        
        html = (wiki.page(key).html())                          # Parsing HTML from wikipedia search of entity 
        soup = BeautifulSoup(html,'html.parser')

        containers = soup.findAll("div",{"class":"mw-parser-output"})  # Getting first paragraph from HTML summary
        text = str(containers[0].find('p',class_=''))

        summary = html2text.HTML2Text()                                # Converting HTML text to plain text
        summary.ignore_links = True
        summary = summary.handle(text)
        
        return summary
    
    
def get_abbreviations(summary):
        
        # 3 methods for extracting abbreviations
        
        pairs = sh.extract_abbreviation_definition_pairs(doc_text=summary)   # method 1 [entity (abbreviation)]
        list1 = list(pairs.keys())

        batRegex = re.compile('\((.*?)\)')                           # method 2  [entity(abbreviation)]
        list2 = re.findall(batRegex,summary)
 
        batRegex = re.compile('\*{2}(.*?)\*{2}')                     # method 3 [entity**abbreviation**] 
        list3 = re.findall(batRegex,summary)                         # HTML2text ->  <b>abc</b> => **abc**

        lists = list1+list2+list3                                    # combining all abbreviations
        lists = list(set(lists))
        
        return lists
    
    
def get_scores(list_,key):
        
        scores = [0]*len(list_)                                # calculating matching scores of all possibilities
        for j in range(len(list_)):
            norm_abb = first_letters(key)
            ext_abb = first_letters(list_[j])
            scores[j]=lvst.jaro_winkler(norm_abb,ext_abb)
            
        return scores
    

In [251]:
res = {}
for i in range(len(loc_keys)):
    try:   
        summary = get_summary(loc_keys[i])
        lists = get_abbreviations(summary)

        new_list1=[]                                              # excluding higher length possibilities
        for j in range(len(lists)):
            if(len(lists[j])<=6):
                new_list1.append(lists[j])

        new_list=[0]*len(new_list1)                               # Removing special characters from abbreviations
        for j in range(len(new_list1)):
            new_list[j] = re.sub('\W+','', new_list1[j])

        for j in range(len(new_list)):                            # adding space between each letters
            new_list[j] = " ".join(new_list[j])

        scores = get_scores(new_list,loc_keys[i])

        if(len(scores)!=0):                                       # Taking highest score abbreviation with cut-off
            index_max = np.argmax(scores)
            if(scores[index_max]>=0.60):
                res[loc_keys[i]] = new_list1[index_max]
            
    except wiki.DisambiguationError as e:                         # Handling disambiguation, no abbreviation
        res[loc_keys[i]] = "--"

In [233]:
res

{'WASHINGTON, D. C': 'D.C.', 'Kansas City, Mo.': 'KCMO', 'LOS ANGELES, CA': 'L.A.'}

In [253]:
res

{'Washington. D.C.': 'D.C.', 'Los Angeles, California': 'L.A.', 'New York, New York': 'NYC'}